# Qwen function call

## 直接使用模型

In [9]:
import openai
import json


openai.api_base = 'http://192.168.0.50:8000/v1'
openai.api_key = 'none'


def call_qwen(messages):
    response = openai.ChatCompletion.create(model='Qwen', messages=messages)
    response = response.choices[0]['message']
    response = json.loads(json.dumps(response, ensure_ascii=False))
    
    print('input:', messages)
    print('output:', response)
    
    return response


if __name__ == '__main__':
    messages = [{'role': 'user', 'content': '你好'}]
    response = call_qwen(messages)


input: [{'role': 'user', 'content': '你好'}]
output: {'role': 'assistant', 'content': '你好！很高兴能为你提供帮助。有什么我可以做的吗？', 'function_call': None}


## function call

参考 [LLM代码实现-Qwen（Function Calling）](https://juejin.cn/post/7330512499173507113)

In [12]:
import openai
import json

openai.api_base = "http://192.168.0.50:8000/v1"
openai.api_key = "none"


def contral(response):
    if response.choices[0].message.function_call:
        result = eval(response.choices[0].message.function_call.arguments)
        switch = "开灯" if result["switch"] else "关灯"

        print(switch)


def call_qwen(messages, functions=None):
    print(messages)
    if functions:
        response = openai.ChatCompletion.create(
            model="Qwen", messages=messages, functions=functions
        )
    else:
        response = openai.ChatCompletion.create(model="Qwen", messages=messages)

    print(response)
    result = response.choices[0].message.function_call
    if result:
        if result.name == "contral":
            contral(response)

    print(response.choices[0].message.content)
    return response


if __name__ == "__main__":
    functions = [
        {
            "name_for_human": "灯光控制",
            "name_for_model": "contral",
            "description_for_model": "灯光控制可以帮助用户开关灯"
                                     + " Format the arguments as a JSON object.",
            "parameters": [
                {
                    "name": "switch",
                    "description": "用于控制灯的开关，若为True则为开灯，若为false则为关灯",
                    "required": True,
                    "schema": {"type": "bool"},
                }
            ],
        }
    ]

    messages = [{"role": "user", "content": "帮我开一下灯"}]
    call_qwen(messages, functions)



[{'role': 'user', 'content': '帮我开一下灯'}]
{
  "model": "Qwen",
  "object": "chat.completion",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u6211\u9700\u8981\u4f7f\u7528\u706f\u5149\u63a7\u5236API\u6765\u6253\u5f00\u706f\u3002",
        "function_call": {
          "name": "contral",
          "arguments": "{\"switch\": True}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "created": 1710506276
}
开灯
我需要使用灯光控制API来打开灯。
